In [59]:
from dotenv import load_dotenv
# OpenAI SDK imports
from agents import Agent, Runner, trace, function_tool
import asyncio
import requests
import os
load_dotenv(override=True)

True

## Newsletter Signup request to customers
Create 3 agents to write a newletter signup email to send to subscribers  
Crete a new agent which will select the best out of the three and send a push notification to us which one was selected  

#### Practive Tooling
Covert the first 3 agents into tools usign OpenAI SDK  
Create one more tool for sending the push notification  
Create a new agent which will use this all 4 tools to generate -> select -> send the email as a push notification.  

In [60]:
# STEP 1

intruction_1 = """You are a sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write professional, serious cold emails.
"""
intruction_2 = """You are a humorous, engaging sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write witty, engaging cols emails that are likely to get a response
"""
intruction_3 = """You are a busy sales agent working for FinAI, a company that provides AI solutions for financial institutions.
You write professional, serious cold emails.
"""

In [61]:

sales_agent_1 = Agent(instructions=intruction_1, name="Professional Sales Agent", model="gpt-4o-mini")
sales_agent_2 = Agent(instructions=intruction_2, name="Humorous Sales Agent", model="gpt-4o-mini")
sales_agent_3 = Agent(instructions=intruction_3, name="Busy Sales Agent", model="gpt-4o-mini")

In [62]:
description = "Write a cold email"

# with trace("Fintech Sales Agent"):
#     results = await asyncio.gather(
#         Runner.run(sales_agent_1, message),
#         Runner.run(sales_agent_2, message),
#         Runner.run(sales_agent_3, message),
#     )
# outputs = [result.final_output for result in results]

# for output in outputs:
#     print(output, "\n\n")

In [63]:
# Converting the agents to tools
tool1 = sales_agent_1.as_tool(tool_name="sales_agent_1", tool_description=description)
tool2 = sales_agent_2.as_tool(tool_name="sales_agent_2", tool_description=description)
tool3 = sales_agent_3.as_tool(tool_name="sales_agent_3", tool_description=description)

In [64]:
# Setup Push notification
def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

In [65]:
# There is one more way to create tools using the @funtion_tool decorator from OPENAI SDK
@function_tool
def send_push_notification(message : str):
    push(message)
    return {"status": "success"}

In [66]:
# Tools can be used in other agents in our example we will try to write an agent which will help us choose the best email
# Also it will send the email to the customer (push notification for simplicity)
tools = [tool1, tool2, tool3, send_push_notification]

instruction = """You are a sales manager at FinAI. Your goal is to find the single best cold sales email using the sales_agent tools.
    Follow these steps carefully:
    1. Genrate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all drafts are ready.
    
    2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
    
    3. Use the send_push_notification tool to send the best email (and only the best email) to the user.
    
    Crucial Rules:
    - You must use the sales agent tools to generate the drafts - do not write them yourself.
    - You must send ONE push notification using the send_push_notification tool - never more than one.
    """
    
selection_agent = Agent(
    name="Sales Agent Manager",
    instructions=instruction,
    model="gpt-4o-mini",
    tools=tools
)

message = "Send a cold email addressed to 'Dear Bank Manager'"

with trace("Sales Manager"):
    result = await Runner.run(selection_agent, message)